In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import random
import pickle
import os
from collections import Counter

# For text processing
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# For deep learning
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# For downloading data
import requests

# For model evaluation
from sklearn.model_selection import train_test_split

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gemwincanete/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
def download_text(url):
    """
    Download text data from a URL.
    """
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to download data: {response.status_code}")
        return None

# URL for Alice's Adventures in Wonderland
alice_url = "https://www.gutenberg.org/files/11/11-0.txt"
text = download_text(alice_url)

# Remove Gutenberg header and footer
if text:
    # Find the start and end of the actual book content
    start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
    end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"
    
    start_idx = text.find(start_marker)
    if start_idx != -1:
        start_idx = text.find("\n", start_idx) + 1
    else:
        start_idx = 0
        
    end_idx = text.find(end_marker)
    if end_idx == -1:
        end_idx = len(text)
    
    # Extract the main content
    text = text[start_idx:end_idx].strip()
    
    # Print a sample of the text
    print(f"Total characters: {len(text)}")
    print("Sample text:")
    print(text[:500], "...")

Total characters: 144599
Sample text:
[Illustration]




Alice’s Adventures in Wonderland

by Lewis Carroll

THE MILLENNIUM FULCRUM EDITION 3.0

Contents

 CHAPTER I.     Down the Rabbit-Hole
 CHAPTER II.    The Pool of Tears
 CHAPTER III.   A Caucus-Race and a Long Tale
 CHAPTER IV.    The Rabbit Sends in a Little Bill
 CHAPTER V.     Advice from a Caterpillar
 CHAPTER VI.    Pig and Pepper
 CHAPTER VII.   A Mad Tea-Party
 CHAPTER VIII.  The Queen’s Croquet-Ground
 CHAPTER IX.    The Mock Turtle’s Story
 CHAPTER X.     The Lobster  ...


In [26]:
def preprocess_text(text):
    """
    Clean and tokenize the text.
    """
    # Convert to lowercase
    text = text.lower()
    
    # Replace newlines with spaces
    text = text.replace('\n', ' ')
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenize into words
    words = word_tokenize(text)
    
    return words

# Preprocess the text
words = preprocess_text(text)

# Print some statistics
print(f"Total number of words: {len(words)}")
print(f"Number of unique words: {len(set(words))}")
print("First 20 words:", words[:20])

# Count word frequencies
word_freq = Counter(words)
print("\nMost common words:")
print(word_freq.most_common(10))

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/gemwincanete/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/Users/gemwincanete/nltk_data'
    - '/Users/gemwincanete/nltk_data'
    - '/Users/gemwincanete/nltk_data'
    - '/Users/gemwincanete/nltk_data'
    - '/Users/gemwincanete/nltk_data'
    - '/Users/gemwincanete/nltk_data'
    - '/Use